# Script #1 

- Takes in Sharepoint Export `OTS_RSVP_45.xlsx`
- Renames column headers to match the Knack Reservation schema `banners_reservations_ots.csv`
- Exports new Reservations table to `output/reservations.csv`
- Makes a new Reservations Locations Table to match schema from `banner_locations.csv`
- Exports new Reservations Locations to `output/reservation_locations.csv`

In [22]:
import pandas as pd
from pandas import read_excel
import numpy as np

df = pd.read_excel("./source_data/OTS_RSVP_45.xlsx", sheet_name="owssvr.dll")

In [25]:
# Setup a new dataframe for Reservations:
df_reservations = df

# Rename columns to match Knack schema
# Sharepoint: OTS_RSVP_45 column names of left
# Knack: banner_reservations_ots on right
column_names = {
    # Sharepoint : Knack
    'ID': 'Sharepoint ID',
    'Permit #': 'Permit Number',
    'EventName': 'Event Name OTS', 
    'Organization': 'Organization Name', 
    # 'Payment Rec?'
    'Delivered?': 'Banners Delivered',
    'Status': 'Banner Status OTS',
    'Quantity': 'Number of Banners OTS',
    # Count Banner Jobs
    'RSVP Date': 'CREATED_DATE',
    'Installation_Date': 'Installation Date OTS',
    'Removal_Date': 'Removal Date OTS',
    # Keep all 'Banner_Face' & 'OTS_Linker' 1-4 columns the same
    # 'App Received?'
    # 'Co-Sponsor?'
    # 'Design Rec?'
    'Will Pick-up?': 'Will Pick Up'
    # 'Item Type'
    # 'Path'
}
df_reservations.rename(columns = column_names, inplace= True)

# TODO: Confirm if we should we remove these unneeded columns?
removed_columns = [
#     'Payment Rec?',
#     'App Received?',
#     'Co-Sponsor?',
#     'Design Rec?',
    'Item Type',
    'Path'
]
df_reservations = df_reservations.drop(columns=removed_columns)



In [26]:
# Export Reservations table to CSV
df_reservations.to_csv(r'./output_data/reservations.csv', index=None, header=True)

In [28]:
# Create a new dataframe for Reservations Locations 
# using headers from the csv
reso_location_column_names = pd.read_csv('./source_data/banner_locations.csv').columns
reso_locations_df = pd.DataFrame(columns=reso_location_column_names)
reso_locations_df['Parent ID'] = None


print('🎲🎲🎲 Reservation Headers')
print(df_reservations.columns)

print('🎲🎲🎲 Reservation Location Headers')
print(reso_locations_df.columns)




🎲🎲🎲 Reservation Headers
Index(['Sharepoint ID', 'Permit Number', 'Event Name OTS', 'Organization Name',
       'Payment Rec?', 'Banners Delivered', 'Banner Status OTS',
       'Number of Banners OTS', 'CREATED_DATE', 'Installation Date OTS',
       'Removal Date OTS', 'Banner_Face_1', 'OTS_Linker_1', 'Banner_Face_2',
       'OTS_Linker_2', 'Banner_Face_3', 'OTS_Linker_3', 'Banner_Face_4',
       'OTS_Linker_4', 'App Received?', 'Co-Sponsor?', 'Design Rec?',
       'Will Pick Up'],
      dtype='object')
🎲🎲🎲 Reservation Location Headers
Index(['Banner Job ID', 'Permit Number', 'banner reservations OT',
       'banners_reservations_LPB', 'banners_district_sub_district',
       'banners_organization', 'street_banners_comment', 'Organization Name',
       'Linker Location', 'Linker Name', 'Direction Facing',
       'Location Installation Date', 'Location Removal Date',
       'OTS Install Date ', 'OTS Remove Date', 'DateEquation', 'Count Total',
       'Status OTS', 'Districts', 'Subdistric

In [29]:
import math

for i in df_reservations.index:
    has_one_location = df_reservations["Number of Banners OTS"][i] == 1
    
    if has_one_location:
        data = {
            # 'Banner Job ID', comes from Knack?
            'Permit Number': df_reservations["Permit Number"][i],
#             'banner reservations OT',
#             'banners_reservations_LPB',
#             'banners_district_sub_district',
            'banners_organization': df_reservations["Organization Name"][i],
#             'street_banners_comment',
            'Organization Name': df_reservations['Organization Name'][i],
            'Linker Location': df_reservations['OTS_Linker_1'][i],
            'Linker Name': df_reservations[f'OTS_Linker_1'][i],
            'Direction Facing': df_reservations['Banner_Face_1'][i],
#             'Location Installation Date',
#             'Location Removal Date',
            'OTS Install Date ': df_reservations['Installation Date OTS'][i],
            'OTS Remove Date': df_reservations['Removal Date OTS'][i],
#             'DateEquation',
            'Count Total': df_reservations['Number of Banners OTS'][i],
            'Status OTS': df_reservations['Banner Status OTS'][i],
#             'Districts',
#             'Subdistricts',
#             'LPB Install Date',
#             'LPB Remove Date',
#             'Status LPB',
#             'Banner Location Comment',
#             'Banner Job ID.1'
        }
    
        # Create a new row
        row_series = pd.Series(data)
        # Add row to dataframe at the current iterator index
        reso_locations_df.loc[i] = row_series

    else:        
        x = 0
        while x < df_reservations["Number of Banners OTS"][i]:
            x += 1
            
            if type(df_reservations["Permit Number"][i]) == float and math.isnan(df_reservations["Permit Number"][i]):
                permit_number = None
                parent_id = None
                
            else:
                parent_id = df_reservations["Permit Number"][i].split(" Permits ")[0].split("Parent ID")[1].strip().rstrip(',')
                permit_list = df_reservations["Permit Number"][i].split(" Permits ")[1].split(",")
                
                permit_number = permit_list[x - 1].strip()
            
            data = {
                # 'Banner Job ID', comes from Knack?
                'Permit Number': permit_number,
    #             'banner reservations OT',
    #             'banners_reservations_LPB',
    #             'banners_district_sub_district',
                'banners_organization': df_reservations["Organization Name"][i],
    #             'street_banners_comment',
                'Organization Name': df_reservations['Organization Name'][i],
                'Linker Location': df_reservations[f'OTS_Linker_{x}'][i],
                'Linker Name': df_reservations[f'OTS_Linker_{x}'][i],
                'Direction Facing': df_reservations[f'Banner_Face_{x}'][i],
    #             'Location Installation Date',
    #             'Location Removal Date',
                'OTS Install Date ': df_reservations['Installation Date OTS'][i],
                'OTS Remove Date': df_reservations['Removal Date OTS'][i],
    #             'DateEquation',
                'Count Total': df_reservations['Number of Banners OTS'][i],
                'Status OTS': df_reservations['Banner Status OTS'][i],
    #             'Districts',
    #             'Subdistricts',
    #             'LPB Install Date',
    #             'LPB Remove Date',
    #             'Status LPB',
    #             'Banner Location Comment',
    #             'Banner Job ID.1',
                'Parent ID': parent_id
            }
            

            # Create a new row
            row_series = pd.Series(data)
            # Add row to dataframe at the current iterator index
            reso_locations_df.loc[f'{i}0{x}'] = row_series               

In [30]:
reso_locations_df

,Banner Job ID,Permit Number,banner reservations OT,banners_reservations_LPB,banners_district_sub_district,banners_organization,street_banners_comment,Organization Name,Linker Location,Linker Name,...,Count Total,Status OTS,Districts,Subdistricts,LPB Install Date,LPB Remove Date,Status LPB,Banner Location Comment,Banner Job ID.1,Parent ID
0,NaN,18-117963,NaN,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#W: 3652 WESTBANK DR.,#W: 3652 WESTBANK DR.,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,19-029392,NaN,NaN,NaN,Spark Cognition,NaN,Spark Cognition,#61: 200 1/2 S CONGRESS AVE,#61: 200 1/2 S CONGRESS AVE,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
201,NaN,18-117958,NaN,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#32: 1799 1/2 S CONGRESS AVE,#32: 1799 1/2 S CONGRESS AVE,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
202,NaN,18-117959,NaN,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#18: 2706 1/2 S LAMAR BLVD,#18: 2706 1/2 S LAMAR BLVD,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
203,NaN,18-117962,NaN,NaN,NaN,"Service Dogs, Inc.",NaN,"Service Dogs, Inc.",#15: 1502 1/2 W 5th ST,#15: 1502 1/2 W 5th ST,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11982651
3,NaN,18-137917,NaN,NaN,NaN,Austin NARI,NaN,Austin NARI,#W: 3652 WESTBANK DR.,#W: 3652 WESTBANK DR.,...,1,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
401,NaN,19-009296,NaN,NaN,NaN,Blanton Museum of Art,NaN,Blanton Museum of Art,# 5: 201 1/2 E MLK BLVD,# 5: 201 1/2 E MLK BLVD,...,2,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12119979
402,NaN,19-009299,NaN,NaN,NaN,Blanton Museum of Art,NaN,Blanton Museum of Art,#64: 2100 1/2 E 7th ST,#64: 2100 1/2 E 7th ST,...,2,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12119979
501,NaN,18-135333,NaN,NaN,NaN,Austin NARI,NaN,Austin NARI,#24: 2800 1/2 S LAMAR BLVD,#24: 2800 1/2 S LAMAR BLVD,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002659
502,NaN,18-135342,NaN,NaN,NaN,Austin NARI,NaN,Austin NARI,#16: 1206 1/2 W 38th ST,#16: 1206 1/2 W 38th ST,...,3,Reservation Input,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12002659


In [33]:
# Export Reservations Locations table to CSV
reso_locations_df.to_csv(r'./output_data/reservation_locations.csv', index=None, header=True)